In [1]:
from zipfile import ZipFile
import pandas as pd
import numpy as np

# Загрузка данных

In [2]:
def gen_parcer(f):
    for line in f.readlines():
        yield line.decode().strip().split("\t")

In [3]:
with ZipFile('data.zip') as datazip:
    with datazip.open('train.tsv') as f:
        pd_train = pd.DataFrame(gen_parcer(f),
                                columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply",
                                         "label", "confidence"])
    with datazip.open('public.tsv') as f:
         pd_public = pd.DataFrame(gen_parcer(f),
                                  columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply"])

In [4]:
pd_train.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.8753516175
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.9009682113
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.8843202145
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.9825304673
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.8380535096


In [5]:
pd_public.head()

,context_id,context_2,context_1,context_0,reply_id,reply
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,2,нет .
3,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,3,"конечно , нет ."
4,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,4,"разумеется , нет ."


In [6]:
pd_train.count()

context_id    97533
context_2     97533
context_1     97533
context_0     97533
reply_id      97533
reply         97533
label         97533
confidence    97533
dtype: int64

In [7]:
pd_public.count()

context_id    9968
context_2     9968
context_1     9968
context_0     9968
reply_id      9968
reply         9968
dtype: int64

In [8]:
target_encoder = {'bad': 0, 'good': 2, 'neutral': 1}  
pd_train.label = pd_train.label.map(target_encoder)

In [9]:
pd_data = pd.concat([pd_train, pd_public], axis=0).reset_index(drop=True)
pd_data.label.fillna(-1, inplace=True)
pd_data.confidence.fillna(0.0, inplace=True)

# Что за данные

In [10]:
pd_train.shape, pd_public.shape, pd_data.shape

((97533, 8), (9968, 6), (107501, 8))

In [11]:
pd_data.groupby("label")["context_id"].count() / len(pd_data)

label
-1.0    0.092725
 0.0    0.323439
 1.0    0.104687
 2.0    0.479149
Name: context_id, dtype: float64

# Простой бейзлайн

In [13]:
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [73]:
import re

In [15]:
w2v_fpath = "w2v/all.norm-sz100-w10-cb0-it1-min100.w2v"
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
w2v.init_sims(replace=True)

In [63]:
for word, score in w2v.most_similar(u"звони"):
    print (word, score)

позвони 0.9512766003608704
позвонишь 0.9257874488830566
позвоню 0.9001246690750122
звякну 0.8846049308776855
звони… 0.8844043016433716
звякни 0.8841025233268738
перезвонишь 0.8796979188919067
перезвони 0.8790677189826965
– звони 0.8747897148132324
— звони 0.8705773949623108


In [20]:
len(w2v.vocab)

1239964

In [115]:
s_words = pd_data[["context_2", "context_1", "context_0", "reply"]] \
    .apply(lambda x: " ".join(x), axis=1)

In [120]:
empty_vector = np.zeros(w2v.vector_size)
def condext_to_vec(x):
    words = re.findall(r"\b(\w+)\b", x)
    vectors = sum([w2v[word] for word in words if word in w2v])
    return (empty_vector + vectors).reshape(1, -1)

In [111]:
to_features_w2v(pd_train).shape

(97533, 200)

In [89]:
s_words.loc[40525]

['сантанико', 'ƒрузь']

In [92]:
w2v['друзь']

array([-0.09578563,  0.01117561, -0.00880318,  0.21853253, -0.01243461,
        0.03876792,  0.03949704, -0.04481348, -0.0351248 ,  0.04386365,
        0.19681863, -0.01494529,  0.02093783, -0.04380528,  0.04432995,
        0.0782617 ,  0.10176197,  0.02426261, -0.05605232, -0.07077625,
       -0.00909789, -0.00149532,  0.16899835,  0.02694597, -0.12898546,
        0.03377159, -0.09571755, -0.06270054,  0.03833026, -0.05506329,
       -0.07291742, -0.11279436,  0.01631826,  0.06522804, -0.04545193,
       -0.16640171, -0.16595216, -0.04210451,  0.06895226,  0.02163207,
       -0.03472768,  0.00218695, -0.05143479, -0.00968446, -0.01106714,
       -0.19546075, -0.06368793,  0.01404457, -0.0222574 , -0.02700514,
        0.05644063,  0.08306466, -0.08505695, -0.04710397, -0.07799276,
        0.00852956, -0.05083572, -0.02046217,  0.14465103,  0.08092348,
        0.1001244 , -0.02005348,  0.04071897, -0.12408392, -0.00646024,
        0.11262947,  0.00634297, -0.07659069,  0.17482547,  0.03

In [123]:
np.concatenate(s_words.apply(condext_to_vec).tolist())

array([[ 0.05916152, -0.64475715, -1.28002584, ...,  1.08181226,
         0.10062046, -1.89331996],
       [ 0.18127358, -0.57715333, -1.30805779, ...,  1.01000834,
         0.21921341, -2.10133481],
       [ 0.06556252, -0.66837418, -1.59541368, ...,  1.29882824,
         0.18910089, -2.44109344],
       ..., 
       [-0.33452025, -0.6943807 , -0.86757255, ...,  0.03578128,
         0.05612306, -1.41522992],
       [-0.34195346, -0.75255901, -0.96607566, ...,  0.13925795,
         0.02872573, -1.5818764 ],
       [-0.32940465, -0.77266765, -0.8980037 , ...,  0.01605068,
         0.0777633 , -1.29312503]])

In [94]:
np.concatenate(s_words.apply(lambda x: (np.zeros(100) + sum((w2v[word] for word in x if word in w2v))).reshape(1, -1)).tolist())

array([[ 0.05916152, -0.64475715, -1.28002584, ...,  1.08181226,
         0.10062046, -1.89331996],
       [ 0.18127358, -0.57715333, -1.30805779, ...,  1.01000834,
         0.21921341, -2.10133481],
       [ 0.06556252, -0.66837418, -1.59541368, ...,  1.29882824,
         0.18910089, -2.44109344],
       ..., 
       [-0.33452025, -0.6943807 , -0.86757255, ...,  0.03578128,
         0.05612306, -1.41522992],
       [-0.34195346, -0.75255901, -0.96607566, ...,  0.13925795,
         0.02872573, -1.5818764 ],
       [-0.32940465, -0.77266765, -0.8980037 , ...,  0.01605068,
         0.0777633 , -1.29312503]])

## features

In [125]:
def to_features_w2v(df):
    s_words = df[["context_2", "context_1", "context_0"]].apply(lambda x: " ".join(x), axis=1)
    context = np.concatenate(s_words.apply(condext_to_vec).tolist())
    
    s_words = df["reply"]
    reply = np.concatenate(s_words.apply(condext_to_vec).tolist())
    
    features = np.concatenate([context, reply], axis=1)
    return features

## crossval

In [97]:
from tqdm import tqdm_notebook

In [98]:
from sklearn.model_selection import StratifiedKFold

In [99]:
skf = StratifiedKFold(n_splits=3)

In [100]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor

In [102]:
from sklearn.metrics import mean_absolute_error

In [103]:
def calc_ndcg_score(df, y_predict):
    df = df.copy()
    df["predict"] = y_predict
    #dcg
    df.sort_values(["context_id", "predict"], ascending=[True, False], inplace=True)
    df["row_count"] = df.groupby("context_id").cumcount()+1
    df["dcg"] = df["label"] / np.log2(df["row_count"] + 1)
    #idcg
    df.sort_values(["context_id", "label"], ascending=[True, False], inplace=True)
    df["row_count"] = df.groupby("context_id").cumcount()+1
    df["idcg"] = df["label"] / np.log2(df["row_count"] + 1)
    
    df_context = df.groupby("context_id").sum()
    return (df_context["dcg"] / df_context["idcg"]).mean() * 100000

In [104]:
def do_cross_val(model_reg, to_features, to_target):
    mae_errors = []
    ndcg_scores = []

    for _train, _test in tqdm_notebook(skf.split(pd_train.index, pd_train.label), total=3):
        i_train = pd_train.iloc[_train]
        i_test = pd_train.iloc[_test]

        X_train = to_features(i_train)
        X_test = to_features(i_test)
        Y_train = to_target(i_train)
        Y_test = to_target(i_test)

        model_reg.fit(X_train, Y_train)
        Y_predict = model_reg.predict(X_test)
        mae_errors.append(mean_absolute_error(Y_test, Y_predict))
        ndcg_scores.append(calc_ndcg_score(i_test, Y_predict))

    print(mae_errors)
    print(ndcg_scores)

## target

In [105]:
trg_label = lambda x: x.label.values

In [106]:
trg_label_confidence = lambda x: (x.label.values - 1) * x.confidence.values.astype(np.float)

## Нормальные варианты

In [127]:
do_cross_val(LinearRegression(), to_features_w2v, trg_label)

A Jupyter Widget


[0.83633135814594051, 0.83542381754126749, 0.83530401366871465]
[86155.36850112038, 85816.3105577912, 86051.75916060706]


In [128]:
do_cross_val(LinearRegression(), to_features_w2v, trg_label_confidence)

A Jupyter Widget


[0.72349039337976706, 0.72178025925071621, 0.7209589339115503]
[86169.43718925674, 85857.15289749001, 86071.59998169028]


## на проверке

In [134]:
from sklearn.ensemble import RandomForestRegressor

In [136]:
do_cross_val(RandomForestRegressor(n_estimators=60, n_jobs=-1), to_features_w2v, trg_label_confidence)

A Jupyter Widget


[0.70617869210100848, 0.70500433760123726, 0.70573993031730142]
[85845.51530003439, 85713.93863663572, 85771.36473734955]


## Неудачные варианты

# Submit

In [81]:
def do_submit(submission_name, model_reg, to_features, to_target):
    i_train = pd_train
    i_test = pd_public.copy()

    X_train = to_features(i_train)
    X_test = to_features(i_test)
    Y_train = to_target(i_train)

    model_reg.fit(X_train, Y_train)
    Y_predict = model_reg.predict(X_test)

    i_test["predict"] = Y_predict
    i_test.sort_values(["context_id", "predict"], ascending=[True, False], inplace=True)

    i_test.head(15)
    i_test[["context_id", "reply_id"]].to_csv(submission_name, index=False, header=False, sep="\t")
    
    return i_test

In [82]:
fit_tfidf(analyzer="char", ngram_range=(3, 4))
do_submit("linear_cv_opsion.tsv", Ridge(alpha=20.0), to_features_tfidf, trg_label_confidence)

,context_id,context_2,context_1,context_0,reply_id,reply,predict
3479,100097508986637,,,выключай их,4,"пытаюсь , сэр .",0.345558
3478,100097508986637,,,выключай их,3,я пытаюсь !,0.162357
3476,100097508986637,,,выключай их,1,"ты не выключишь , тогда € сам выключу",-0.021206
3480,100097508986637,,,выключай их,5,выключите,-0.270745
3475,100097508986637,,,выключай их,0,выключить что ?,-0.284118
3477,100097508986637,,,выключай их,2,выключить ?,-0.332019
3483,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,2,я не могу позволить вам уйти .,0.304185
3482,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,1,"мне показалось , вы меня зовете .",0.258659
3484,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,3,мне уже легче .,0.219066
3486,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,5,"что "" не надо "" ? не надо меня .",0.054430
